# Reconstructing Hourly Energy Availability & Reserve Margin

- **Thermal Resources:** Use `Provide_Power_Capacity_In_Timepoint_MW`
- **Wind, Solar:** Use `Provide_Power_Capacity_In_Timepoint_MW`, making the claim that curtailment is only an economic decision & not a reliability one (i.e., if energy were needed, renewables could be un-curtailed)
- **Geothermal, Biomass & Hydro:** Use `Provide_Power_MW`, since these have implicit and/or explicit energy limits that may/may not be reflected just by using `Provide_Power_Capacity_In_Timepoint_MW`
- **Storage:** Using just `Provide_Power_MW` does not reflect that storage has energy in storage even in hours that it is not being dispatched. Instead, derive a metric from available state-of-charge in each hour, and there are a few options:
    1. $`SOC_Inter_Intra_Joint` / `duration`$: Define energy availability as MW that the storage could discharge at for its entire duration (e.g., an 100 MW, 8-hour storage with 400 MWh SoC in a given hour would have an effective hourly rating of 50 MW)
    2. $min(`Provide_Power_Capacity_In_Timepoint_MW`, `SOC_Inter_Intra_Joint`)$: Doesn't matter that the storage can't discharge for multiple hours, if it has enough SoC for a given hour, that's it's energy availability
- **Imports:** If you [read the I&A,§7.1.13 Imports](https://www.cpuc.ca.gov/-/media/cpuc-website/divisions/energy-division/documents/integrated-resource-plan-and-long-term-procurement-plan-irp-ltpp/2023-irp-cycle-events-and-materials/inputs-assumptions-2022-2023_final_document_10052023.pdf), a difference in assumption between `resolve` and `servm` is that the `servm` model assumes a hourly shape for imports that can be relied on for RA. So there are actually three different hourly imports you could use:
    1. The `resolve` actual hourly dispatch (tx limits for economic dispatch are less constrained than the 11,040 MW/5,000 MW assumed
    2. The `resolve` assumed RA imports (which is for some reason that I don't remember 3,100 MW in 2045)
    3. The `servm` assumed shaped RA imports (11,040 MW, derated to 5,000 MW during summer evenings)

In [ ]:
import pandas as pd
import upath

In [2]:
base_path = upath.UPath("../2045-Core_test/Baseline/2045-2_45_CESM2_2015/2024-08-16 10-48-13")

In [3]:
# Variable = gen + curtailment
# Storage = SoC / duration
# Thermal = capacity

hourly_capacity = pd.read_csv(base_path / "expressions" / "Plant_Provide_Power_Capacity_In_Timepoint_MW.csv", index_col=[0, 1, 2, 3])
hourly_gen = pd.read_csv(base_path / "variables" / "Provide_Power_MW.csv", index_col=[0, 1, 2, 3])
hourly_soc = pd.read_csv(base_path / "expressions" / "SOC_Inter_Intra_Joint.csv", index_col=[0, 1, 2, 3])
hourly_curtailment = pd.read_csv(base_path / "expressions" / "Scheduled_Curtailment_MW.csv", index_col=[0, 1, 2, 3])
hourly_tx = pd.read_csv(base_path / "variables" / "Transmit_Power_Forward_MW.csv", index_col=[0, 1, 2, 3])
hourly_load = pd.read_csv(base_path / "parameters" / "input_load_mw.csv", index_col=[0, 1, 2, 3])

caiso_resources = pd.read_csv(base_path / "results_summary" / "resource_summary.csv", index_col=0)
caiso_resources = caiso_resources.loc[(caiso_resources["Operational Capacity (MW)"] != 0) & (caiso_resources["Zone"] == "CAISO"), :].index.tolist()

caiso_paths = [tx for tx in hourly_tx.index.get_level_values(0).unique() if "to CAISO" in tx]

In [5]:
merged = pd.concat(
    [
        hourly_capacity.loc[hourly_capacity.index.get_level_values(0).isin(caiso_resources), :],
        hourly_gen.loc[hourly_gen.index.get_level_values(0).isin(caiso_resources), :],
        hourly_soc.loc[hourly_soc.index.get_level_values(0).isin(caiso_resources), :],
        # hourly_curtailment.loc[hourly_curtailment.index.get_level_values(0).isin(caiso_resources), :],
        # hourly_tx.loc[hourly_tx.index.get_level_values(0).isin(caiso_paths), :],
    ],
    axis=1
)
merged.to_csv("./merged.csv", index=True)

# Questions for Liyang
- [ ] What changes did Liyang make to `provide_power_potential_profile`, because it's different than I expected for the gas resources?
- [ ] What is the `reservemargin` subfolder?